In [1]:
%cd ~/corpus

/home/sibava/corpus


In [2]:
import json
import jsonlines
import pandas as pd
from transformers import T5Tokenizer
from pprint import pprint
import copy
import itertools
from tqdm.auto import tqdm

In [3]:
tokenizer = T5Tokenizer.from_pretrained('megagonlabs/t5-base-japanese-web')

In [4]:
psa_df = pd.read_json("/home/sibava/corpus/psa_instanceC.jsonl",orient='records',lines=True)
outputfile = open("/home/sibava/corpus/psat5instance.jsonl",mode='w')

In [5]:
psa_df['arg_surface'].mask(psa_df['arg_type'] == 'exog','<extra_id_99>',inplace=True)
psa_df['arg_surface'].mask(psa_df['arg_type'] == 'exo1','<extra_id_98>',inplace=True)
psa_df['arg_surface'].mask(psa_df['arg_type'] == 'exo2','<extra_id_97>',inplace=True)

In [6]:
ga_df = psa_df.iloc[::3,].reset_index(drop=True)
o_df = psa_df.iloc[1::3,].reset_index(drop=True)
ni_df = psa_df.iloc[2::3,].reset_index(drop=True)

In [7]:
for df,case in zip([ga_df,o_df,ni_df],['が','を','に']):
	df['arg_surface'].mask(df['arg_type'] == 'none','',inplace=True)
	df['arg_surface'].where(df['arg_type'] == 'none',df['arg_surface'] + case,inplace=True)

In [8]:
input_ids = []
for context,pred_sent_index,pred_indices in zip(ga_df['context'].values,ga_df['pred_sent_index'].values,ga_df['pred_indices'].values):
	pred_h = pred_indices[0]
	pred_t = pred_indices[-1]
	input_text = context[pred_sent_index][:pred_h] + ["<extra_id_0>"] + context[pred_sent_index][pred_h:pred_t +1] + ["<extra_id_1>"] + context[pred_sent_index][pred_t +1:]
	concated_context = ''.join(list(itertools.chain.from_iterable(input_text)))
	padded_input_ids = tokenizer.encode(concated_context,max_length=512,padding="max_length")
	truncated_input_ids = padded_input_ids[len(padded_input_ids)-512:]
	input_ids.append(truncated_input_ids)

In [9]:
label_ids = [] 
labels = ga_df['arg_surface'].str.cat(o_df['arg_surface'].str.cat(ni_df['arg_surface'].str.cat(ga_df['pred_surface'])))
for label in labels.values:
	label_ids.append(tokenizer(label,max_length=30,padding="max_length").input_ids)

In [49]:
gold_arguments = []
for ga_goldchain,o_goldchian,ni_goldchian in zip(ga_df['goldchain'].values,o_df['goldchain'].values,ni_df['goldchain'].values):
	gold_argument = {
		'ga':ga_goldchain,
		'o':o_goldchain,
		'ni':ni_goldchain
	}	
	gold_arguments.append(gold_argument)

In [39]:
arg_types = []
for ga_arg_type,o_arg_type,ni_arg_type in zip(ga_df['arg_type'].values,o_df['arg_type'].values,ni_df['arg_type'].values):
	arg_type = {
		'ga':ga_arg_type,
		'o':o_arg_type,
		'ni':ni_arg_type
	}	
	arg_types.append(arg_type)
	

In [42]:
t5_df = pd.DataFrame({
	"input_ids":input_ids,
	"label_ids":label_ids,
	"gold_arguments":gold_arguments,
	"arg_types":arg_types,
	"alt_type":ga_df['alt_type']
})

In [50]:
with open("/home/sibava/corpus/psat5instance.jsonl",mode='w') as outputfile:
	t5_df.to_json(outputfile,orient='records',lines=True)